In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import os
import importlib

from kaggle_submit import submit_to_kaggle
from helpers import *
from utils.globals import *
from utils.distribution_statistics import *

train_file = "kaggle_data/X_train.h5/X_train.h5"
test_file = "kaggle_data/X_test.h5/X_test.h5"

h5_train = h5py.File(train_file, mode='a')
h5_test = h5py.File(test_file, mode='a')

y_train = pd.read_csv("kaggle_data/y_train.csv", index_col=0, squeeze=True)
y_train_arr = y_train.to_numpy()

# MAKE CUSTOM FEATURES
from additional_features.make_features import make_all_features
make_all_features(h5_train, h5_test, n_chunks=10, verbose=True, overwrite=True)


from objects import *


_create_eeg_mean
_create_log_energy


/Users/theophile/Desktop/ECP/3A/ML/project/additional_features/eeg_band_log_energies.py:70: RuntimeWarning: overflow encountered in log
  log_energy = np.log(1 + energy)


_create_log_modulus


/Users/theophile/Desktop/ECP/3A/ML/project/additional_features/features_to_frequential.py:62: RuntimeWarning: overflow encountered in log
  1 + get_spectrum_modulus(h5_file[orig_feat][chunk_start : chunk_end], sampling_freq)


_create_pulse_max_log_energy_and_freq


/Users/theophile/Desktop/ECP/3A/ML/project/additional_features/pulse_to_freq.py:47: RuntimeWarning: overflow encountered in log
  h5_file["pulse_max_logE"][chunk_start:chunk_end] = np.log(1 + max_energies)[:, None]


_create_speed_and_acceleration


In [2]:
features_df = pd.DataFrame(list(set(h5_train.keys()) - set(IRRELEVANT_FEATURES)), columns=["Feature"])
features_df.loc[:, "Dim"] = features_df['Feature'].apply(lambda x: h5_train[x][0].shape[0])
features_df.sort_values(by=["Dim", "Feature"])

import re

BAND_LOG_ENERGY_FEATURES = [feat for feat in FEATURES if re.search("(?:(?:alpha)|(?:beta)|(?:delta)|(?:theta)){1}.*_logE", feat)]

SLEEP_FEATURES = [feat for feat in FEATURES if re.search('sleep.*[^(?:logmod)]', feat)]

LOGMOD_FEATURES = [feat for feat in FEATURES if re.search('.*_logmod', feat)]

TIME_FEATURES

OTHER_FEATURES = ["pulse_max_freq", "pulse_max_logE"]

_features = sum([BAND_LOG_ENERGY_FEATURES, SLEEP_FEATURES, LOGMOD_FEATURES, TIME_FEATURES, OTHER_FEATURES], [])
assert sorted(_features) == sorted(FEATURES)



In [156]:
def make_input_default(h5_file):
    df_bandlog = make_input_new(
        h5_file,
        features=BAND_LOG_ENERGY_FEATURES,
        rescale_by_subject=False,
        moments=[1],
        #pre_op=np.exp,
        #pre_op_name="energy"
    )
    
    df_sleep = make_input_new(
        h5_file,
        features=SLEEP_FEATURES,
        rescale_by_subject=False,
        moments=[1]
    )
    
    ## LOGMOD RENAME COLUMNS
    
    df_logmod = make_input_new(
        h5_file,
        features=LOGMOD_FEATURES,
        rescale_by_subject=False,
        # moments=[1],
        quantiles_inv=[0.1, 0.3, 0.5, 0.7, 0.9],
        diff_orders=[0],
        #pre_op=lambda x: np.exp(2 * x),
        #pre_op_name="energy"
    )
    
   # cols_no_rescale = [(col[0] + "_no_rescale", *col[1:]) for col in df_logmod_no_rescale.columns]
   # df_logmod_no_rescale.columns = pd.MultiIndex.from_tuples(cols_no_rescale)
    
   # df_logmod_with_rescale = make_input_bis(
   #     h5_file,
   #     features=LOGMOD_FEATURES,
   #     rescale=True,
   #     moments=[1, 2],
   #     quantiles=[0.05, 0.95],
   #     quantiles_inv=[0.1, 0.3, 0.5, 0.7, 0.9],
   #     diff_orders=[0, 1],
   #     pre_op=lambda x: np.exp(2 * x),
   #     pre_op_name="energy"
   # )
    
   # cols_with_rescale = [(col[0] + "_with_rescale", *col[1:]) for col in df_logmod_with_rescale.columns]
   # df_logmod_with_rescale.columns = pd.MultiIndex.from_tuples(cols_with_rescale)
    
    ## END LOGMOD RENAME COLUMNS
    
    
    df_time_diff_0 = make_input_new(
        h5_file,
        features=TIME_FEATURES,
        rescale_by_subject=False,
        # moments=[1, 2],
        quantiles=[1e-4, 0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 0.99, 1-1e-4],
        diff_orders=[0]
    )
    
    #df_time_diff_1 = make_input_new(
    #    h5_file,
    #    features=TIME_FEATURES,
    #    rescale_by_subject=False,
    #    # moments=[1, 2],
    #    quantiles=[1e-4, 1-1e-4],
    #    diff_orders=[1]
    #)
    
    #df_pulse_max_freq = make_input_new(
    #    h5_file,
    #    features=["pulse_max_freq"],
    #    rescale_by_subject=True,
    #    moments=[1],
    #)
    
    #df_pulse_max_logE = make_input_new(
    #    h5_file,
    #    features=["pulse_max_logE"],
    #    rescale_by_subject=False,
    #    moments=[1],
        #pre_op=np.exp,
        #pre_op_name="energy"
    #)
    
    
    return pd.concat([
        df_bandlog,
        df_sleep,
        df_logmod, 
        #df_logmod_with_rescale, 
        df_time_diff_0,
        #df_time_diff_1,
        #df_pulse_max_freq,
        #df_pulse_max_logE,
        ],
        axis=1)

def shift_and_fill(df, shift):
    shifted_df = df.shift(shift)
    if shift > 0:
        shifted_df.bfill(inplace=True)
    elif shift < 0:
        shifted_df.ffill(inplace=True)
    return shifted_df


def roll_and_concat(df, shifts_range):
    return pd.concat(map(lambda shift: shift_and_fill(df, shift), shifts_range), 
                     axis=1, keys=shifts_range)    
    
def concat_windows(h5_file, df, shifts):
    df = df.groupby(h5_file["index"][:], as_index=False).apply(roll_and_concat, shifts_range=shifts)
    return df
    
def make_input_default_test(h5_file):
    return make_input_new(h5_file, ["eeg_1", "eeg_2"], moments=[1])

def make_input_default_rolling(h5_file, shifts):
    """
    !!! not suited for pca because columns have 3 levels
    """
    df = make_input_default(h5_file)
    df_with_window = concat_windows(h5_file, df, shifts)
    return df_with_window


In [157]:
X_train_raw, X_test_raw = make_input_default(h5_train), make_input_default(h5_test)

In [158]:
train_ids = get_subject_ids(h5_train)
train_train_ids, train_val_ids = train_ids[:28], train_ids[28:]

X_train_train = X_train_raw.loc[subjects_ids_to_indexers(h5_train, train_train_ids, as_indices=True), :]
y_train_train = y_train_arr[subjects_ids_to_indexers(h5_train, train_train_ids, as_indices=True)]

X_train_val = X_train_raw.loc[subjects_ids_to_indexers(h5_train, train_val_ids, as_indices=True), :]
y_train_val = y_train_arr[subjects_ids_to_indexers(h5_train, train_val_ids, as_indices=True)]

"""
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

scaler_ = MinMaxScaler()
pca_ = PCA(0.99)

X_train_train = pca_.fit_transform(scaler_.fit_transform(X_train_train))
X_train_val = pca_.transform(scaler_.transform(X_train_val))
X_test = pca_.transform(scaler_.transform(X_test_raw))
"""
def subjects_ids_col(h5_file):
    return h5_file["index"][:]

def concat_windows(arr, subjects_ids, h5_file, shifts): # subjects_ids must be sorted
    sid_col = subjects_ids_col(h5_file)
    sid_col = sid_col[np.isin(sid_col, subjects_ids)]
    df = pd.DataFrame(arr)
    
    return df.groupby(sid_col).apply(roll_and_concat, shifts_range=shifts)

shifts = [-1, 0, 1]
X_train_train_rolled = concat_windows(X_train_train, train_train_ids, h5_train, shifts)
X_train_val_rolled = concat_windows(X_train_val, train_val_ids, h5_train, shifts)




In [171]:
from sklearn.ensemble import RandomForestClassifier

estimator_rf = RandomForestClassifier(
    # verbose=1,
    random_state=1,
    # max_depth=10,
    n_estimators=200, # default=100
    # min_samples_leaf=10,
)

estimator_rf.fit(X_train_train_rolled, y_train_train)

train_score_rf = custom_score(estimator_rf.predict(X_train_train_rolled), y_train_train)
val_score_rf = custom_score(estimator_rf.predict(X_train_val_rolled), y_train_val)

In [172]:
print(train_score_rf)
print(val_score_rf)

0.655940496519377
0.6360466510558397


| Random Forest Params | Time Features Quantiles | Time Features Moments | Sleep Features | Pulse Freq (f_max, A_max) | Shifts | Comments | Training Score | Validation Score |
| :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |
| None | 0.1, 0.5, 0.9 | None | No | None | 0 | - | 1| 0.67|
| None | 0.1, 0.5, 0.9 | None | Yes | None | 0 | - | 1 | 0.69|
| None | 0.1, 0.5, 0.9 | None | Yes | None | -1, 0, 1 | - | 1 | 0.7|
| None | 0.01, 0.1, 0.5, 0.9, 0.99 | None | Yes | None | -1, 0, 1 | - | 1| 0.7  |
| `min_samples_leaf=10` | 0.01, 0.1, 0.5, 0.9, 0.99 | None | Yes | None | -1, 0, 1 | - | 0.89 | 0.69  |
| `min_samples_leaf=10` | 0.01, 0.1, 0.5, 0.9, 0.99 | 1, 2 | Yes | None | -1, 0, 1 |  - | 0.89 | 0.69  |
| None | 0.01, 0.1, 0.5, 0.9, 0.99 | 1, 2 | Yes | None | -1, 0, 1 |  - | 1 | 0.697  |
| None | 0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 0.99 | None | Yes | None | -1, 0, 1 |  - | 1 | 0.708  |
| None | 0.01, DECILES, 0.99 | None | Yes | None | -1, 0, 1 |  - | 1 | 0.709  |
| `min_samples_leaf=10` | 0.01, DECILES, 0.99 | None | Yes | None | -1, 0, 1 |  - | 0.89 | 0.697 |
| `min_samples_leaf=10` | MIN, 0.01, ODD_DECILES, 0.99, MAX | None | Yes | None | -1, 0, 1 |  - | 0.89 | 0.699 |
| None | MIN, 0.01, ODD_DECILES, 0.99, MAX | None | Yes | None | -1, 0, 1 |  - | 1 | 0.713929 |
| None | MIN, 0.01, ODD_DECILES, 0.99, MAX | None | Yes | Yes | -1, 0, 1 |  - | 1 | 0.7 |
| `min_samples_leaf=10` | MIN, 0.01, ODD_DECILES, 0.99, MAX | None | Yes | Yes | -1, 0, 1 | - | 0.89 | 0.697 |
| `min_samples_leaf=10` | MIN, 0.01, ODD_DECILES, 0.99, MAX | None | Yes | Pulse Only | -1, 0, 1 | - | 0.89 | 0.7 |
| None | MIN, 0.01, ODD_DECILES, 0.99, MAX | None | Yes | Pulse Only | -1, 0, 1 |  - | 1 | 0.7055 |
| None | MIN, 0.01, ODD_DECILES, 0.99, MAX | None | Yes | None | -1, 0, 1 | - |  1 | 0.7055 |
| None | MIN, 0.01, ODD_DECILES, 0.99, MAX + derivee 0.5 | None | Yes | None | -1, 0, 1 |  - | 1 | 0.708 |
| `min_samples_leaf=10` | MIN, 0.01, ODD_DECILES, 0.99, MAX + derivee 0.5 | None | Yes | None | -1, 0, 1 |  - | 0.89 | 0.700|
| `min_samples_leaf=10` | MIN, 0.01, ODD_DECILES, 0.99, MAX + derivee MIN, MAX | None | Yes | None | -1, 0, 1 | - | 0.89| 0.7 |
| None | MIN, 0.01, ODD_DECILES, 0.99, MAX + derivee MIN, MAX | None | Yes | None | -1, 0, 1 | - | 1| 0.703|
| None | MIN, 0.01, ODD_DECILES, 0.99, MAX | None | Yes | None | -1, 0, 1 | `bandlog rescaled`| 1| 0.665 |
| None | MIN, 0.01, ODD_DECILES, 0.99, MAX | None | Yes | None | -1, 0, 1 | `quantiles_inv = 10%, 90% for logmod`| 1| 0.7165 |
| `min_samples_leaf=10` | MIN, 0.01, ODD_DECILES, 0.99, MAX | None | Yes | None | -1, 0, 1 | `quantiles_inv = 10%, 90% for logmod`| ? | 0.70 < x < 0.71 |
| None | MIN, 0.01, ODD_DECILES, 0.99, MAX | None | Yes | None | -1, 0, 1 | `quantiles_inv = ODD_DECILES for logmod`| 1 | 0.7371780518172594 |
| `min_samples_leaf=10` | MIN, 0.01, ODD_DECILES, 0.99, MAX | None | Yes | None | -1, 0, 1 | `quantiles_inv = ODD_DECILES for logmod`| 0.898 | 0.721 |

In [150]:
test_ids = get_subject_ids(h5_test)
X_test_rolled = concat_windows(X_test_raw, test_ids, h5_test, shifts)
y_pred = estimator_rf.predict(X_test_rolled)
#submit_to_kaggle(y_pred, h5_test, fname='rf_best_alternate_2021-01-01.csv', msg="")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished


New submission file at submissions/rf_best_alternate_2021-01-01.csv


In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

scaler_ = MinMaxScaler()

X_train_train_rolled_svc = scaler_.fit_transform(X_train_train_rolled) 
X_train_val_rolled_svc = scaler_.transform(X_train_val_rolled)
X_test_rolled_svc = scaler_.transform(X_test_rolled)


estimator_svc = SVC(verbose=1, kernel='rbf', C=1, max_iter=1000, random_state=1)
estimator_svc.fit(X_train_train_rolled_svc, y_train_train)

train_score_svc = custom_score(estimator_svc.predict(X_train_train_rolled_svc), y_train_train)
val_score_svc = custom_score(estimator_svc.predict(X_train_val_rolled_svc), y_train_val)

In [44]:
X_test_rolled.shape

(24980, 63)

In [45]:
X_train_val_rolled.shape

(1988, 243)

In [165]:
X_train_val_rolled.shape

(1988, 513)

In [2]:
import numpy as np

In [4]:
np.add((np.array([[1, 2], [3, 4]]), np.array([[2, 3], [3, 4]])))

22

In [10]:
import re
[feat for feat in FEATURES + ["eeg_mean"] if re.search("^eeg_(?:\d|(?:mean)){1}$", feat)]

['eeg_5', 'eeg_6', 'eeg_3', 'eeg_1', 'eeg_2', 'eeg_7', 'eeg_4', 'eeg_mean']

KeyError: "Couldn't delete link (callback link pointer is NULL (specified link may be '.' or not exist))"